## Import Libraries

In [47]:
import pandas as pd

## Import Data

In [48]:
df_main = pd.read_csv('data/rishi_stores.csv')
df_reference = pd.read_csv('data/safegraph-dataset.csv')

In [80]:
def checkMain():
    main_dict = {}
    black_list = {}
    for i in range(len(df_main)):
        new_key = df_main.loc[i, "zip_code"]
        new_value = df_main.loc[i, "address"]
        if new_key not in main_dict.keys() and new_key not in black_list.keys():
            main_dict[new_key] = new_value
        elif new_key not in black_list.keys():
            if main_dict[new_key] != new_value:
                main_dict.pop(new_key)
                black_list[new_key] = ''
    return main_dict, black_list
main_dict, main_black_list = checkMain()

In [81]:
def checkReference():
    reference_dict = {}
    black_list = {}
    for i in range(len(df_reference)):
        new_key = df_reference.loc[i, "postal_code"]
        new_value = df_reference.loc[i, "placekey"]
        if new_key not in main_dict.keys() and new_key not in black_list.keys():
            main_dict[new_key] = new_value
        elif new_key not in black_list.keys():
            if main_dict[new_key] != new_value:
                main_dict.pop(new_key)
                black_list[new_key] = ''
    return main_dict, black_list
ref_dict, ref_black_list = checkReference()

In [82]:
def checkAddresses():
    address_dict = {}
    for i in range(len(df_reference)):
        new_key = df_reference.loc[i, "street_address"]
        new_value = df_reference.loc[i, "placekey"]
        address_dict[new_key] = new_value
    return address_dict
address_dict = checkAddresses()

In [83]:
for i in range(len(df_main)):
    zip_code = df_main.loc[i, "zip_code"]
    if zip_code not in ref_black_list:
        df_main.loc[i, "placekey"] = ref_dict[zip_code]

KeyError: 33870

In [66]:
count = 0 
for i in range(len(df_main)):
    address = df_main.loc[i, "address"]
    if address in address_dict:
        zip_code = address_dict[address]
        df_main.loc[i, "placekey"] = zip_code
        count += 1
print(count)

0


In [78]:
df_main.head()

,store_skey,city,state_code,zip_code,address,placekey
0,1061,Lecanto,FL,34461,2932 W. Norvell Bryant Highway,2932 W. Norvell Bryant Highway
1,215,Norfolk,VA,23502,1146 N Military Hwy.,NaN
2,498,Philadelphia,PA,19137,3901 Aramingo Ave,NaN
3,804,Kissimmee,FL,34747,7940 West Irlo Bronson Hwy,7940 West Irlo Bronson Hwy
4,704,King of Prussia,PA,19406,145 W. DeKalb Pike,NaN


In [86]:
print(len(main_black_list)) 
print(len(ref_black_list))

220
582


In [87]:
print(main_black_list)

{21061: '', 33578: '', 19406: '', 34744: '', 19152: '', 19114: '', 23502: '', 32837: '', 32822: '', 19033: '', 32806: '', 8054: '', 8052: '', 33612: '', 8083: '', 19063: '', 32707: '', 33619: '', 8002: '', 32809: '', 23005: '', 21921: '', 8527: '', 23704: '', 19115: '', 21224: '', 18954: '', 8753: '', 19709: '', 8857: '', 8406: '', 34207: '', 8050: '', 8053: '', 19355: '', 8021: '', 19007: '', 23834: '', 19044: '', 19107: '', 19137: '', 19147: '', 19335: '', 19382: '', 8055: '', 18944: '', 19901: '', 22191: '', 18106: '', 8080: '', 22406: '', 22407: '', 7753: '', 8037: '', 19342: '', 34787: '', 19525: '', 19104: '', 23831: '', 8003: '', 32703: '', 8865: '', 22554: '', 22408: '', 8244: '', 33837: '', 34994: '', 19111: '', 8087: '', 8203: '', 8077: '', 19610: '', 22911: '', 8094: '', 19904: '', 19061: '', 8332: '', 8361: '', 8016: '', 23518: '', 33317: '', 19446: '', 22401: '', 19090: '', 8831: '', 7731: '', 23320: '', 8701: '', 18103: '', 8012: '', 19702: '', 19428: '', 23452: '', 8902:

In [88]:
print(ref_black_list)

{21009: '', 22485: '', 33803: '', 32608: '', 8007: '', 33065: '', 19010: '', 34234: '', 34669: '', 8824: '', 23294: '', 23185: '', 33542: '', 20110: '', 18322: '', 19095: '', 21401: '', 32073: '', 32601: '', 32901: '', 8735: '', 19958: '', 23692: '', 33544: '', 33556: '', 34211: '', 8098: '', 20646: '', 33063: '', 32225: '', 19805: '', 19153: '', 32940: '', 33634: '', 32746: '', 33186: '', 18301: '', 8753: '', 20016: '', 19610: '', 8618: '', 32829: '', 32966: '', 19111: '', 7054: '', 7840: '', 8016: '', 32819: '', 20010: '', 8049: '', 33157: '', 33710: '', 8837: '', 19008: '', 7004: '', 8055: '', 23235: '', 20754: '', 19604: '', 19901: '', 19473: '', 21286: '', 20619: '', 33312: '', 22180: '', 32804: '', 8872: '', 8089: '', 19015: '', 33781: '', 7109: '', 7032: '', 19050: '', 19107: '', 8110: '', 34293: '', 19438: '', 34747: '', 7203: '', 19056: '', 23233: '', 32832: '', 34471: '', 7731: '', 23502: '', 19002: '', 8068: '', 21244: '', 32822: '', 32763: '', 21613: '', 19382: '', 19317: '